In [1]:
import os
import random
from collections import Counter
from datasets import load_dataset
from together import Together
import pandas as pd

/home/harshil/miniconda3/envs/rllms_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv

load_dotenv()

# # Initialize Together client
os.environ["TOGETHER_API_KEY"] ="3116fd3668302432d738187aa87a1ef3d5f89559b73d736b02f31da487baebd0"
client = Together()

# Set Together AI API Key
# os.environ["TOGETHERAI_API_KEY"] = os.getenv("TOGETHER_API_KEY")

# Load a random subset of 3 questions from the validation set
dataset = load_dataset("commonsense_qa", split="validation")
sampled_dataset = random.sample(list(dataset), 100)

def format_prompt(question, choices):
    choice_str = "\n".join([f"{label}. {text}" for label, text in zip(choices['label'], choices['text'])])
    return f"Question: {question}\nChoices:\n{choice_str}\nAnswer with the correct letter. Give explaination in the next line."

def get_response(prompt, temperature=0.0):
    response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

def self_consistent_answer(question, choices, n_samples=3):
    prompt = format_prompt(question, choices)
    answers = [get_response(prompt, temperature=0.7) for _ in range(n_samples)]
    # print("Prompt:")
    # print(prompt)
    # print("Answers:")
    # for answer in answers:
        # print("------")
        # print(answer)
        # print("------")

    return Counter(answers).most_common(1)[0][0],answers

# Evaluate self-consistency
self_consistency_accuracy = 0
greedy_accuracy = 0
difference = 0
better = 0
worse = 0


In [3]:
df = pd.DataFrame(columns=['Question', 'Answer Match Greedy', 'Answer Match Self-consistency', 'Correct Answer', 'Greedy Answer', 'Self Answer','Approaches Agree'])

In [4]:

for idx, item in enumerate(sampled_dataset, 1):
    if item["question"] in df['Question'].values:
        print(f"Skipping question {idx} as it is already in the dataframe.")
        continue

    print(f"\n--- Question {idx} ---")
    df.loc[idx, 'Question'] = item["question"]
    greedy_answer = get_response(format_prompt(item["question"], item["choices"]), temperature=0.0)
    df.loc[idx, 'Greedy Answer'] = greedy_answer

    predictions = []

    n = 4
    prediction, answers = self_consistent_answer(item["question"], item["choices"], n_samples=n)
    df.loc[idx, 'Self Answer'] = prediction
    correct_answer = item["answerKey"] 
    index = item['choices']['label'].index(item['answerKey'])
    correct_answer = correct_answer + ". " + item['choices']['text'][index]    

    df.loc[idx, 'Correct Answer'] = correct_answer

    if correct_answer == greedy_answer[0]:
        df.loc[idx, 'Answer Match Greedy'] = True
        greedy_accuracy += 1
    else:
        df.loc[idx, 'Answer Match Greedy'] = False
    
    if correct_answer == prediction[0]:
        df.loc[idx, 'Answer Match Self-consistency'] = True
        self_consistency_accuracy += 1
    else :
        df.loc[idx, 'Answer Match Self-consistency'] = False
    
    if greedy_answer.split('\n')[0][0] != prediction.split('\n')[0][0]:
        df.loc[idx, "Approaches Agree"] = False
        difference += 1
        if prediction[0] == correct_answer:
            better += 1
        if greedy_answer[0] == correct_answer:
            worse += 1
    else:
        df.loc[idx, 'Approaches Agree'] = True

    df.to_csv("mistral_7b_instruct_output.csv")

print("--- Evaluation ---")
print("self_consistency_accuracy ", self_consistency_accuracy, "%")
print("greedy_accuracy ", greedy_accuracy, "%")
print("difference ", difference, "%")
print("better ", better, "%")
print("worse ", worse, '%')


--- Question 1 ---

--- Question 2 ---

--- Question 3 ---

--- Question 4 ---

--- Question 5 ---

--- Question 6 ---

--- Question 7 ---

--- Question 8 ---

--- Question 9 ---

--- Question 10 ---

--- Question 11 ---

--- Question 12 ---

--- Question 13 ---

--- Question 14 ---


KeyboardInterrupt: 